<a href="https://colab.research.google.com/github/spolivin/cifar10-website/blob/master/nn_dev/training_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning files

In [ ]:
!git clone https://github.com/spolivin/cifar10-website.git

Cloning into 'cifar10-website'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 4), reused 24 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 9.40 KiB | 4.70 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
%cd cifar10-website/nn_dev
%ls

/content/cifar10-website/nn_dev
pytorch_models/  pytorch_trainer/


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from pytorch_trainer.trainer import Trainer
from pytorch_models import ednet4, resnet20

BATCH_SIZE = 128

In [ ]:
train_transform = transforms.Compose(
    [
        # Data Augmentation (before normalization)
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        # Conversion to tensor
        transforms.ToTensor(),
        # Normalization (after augmentation)
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ]
)

In [ ]:
# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    transform=train_transform,
    download=True,
)

other_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    transform=test_transform,
    download=True,
)

100%|██████████| 170M/170M [00:03<00:00, 43.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#training_set = torch.utils.data.Subset(training_set, torch.arange(10000))
validation_set = torch.utils.data.Subset(other_set, torch.arange(1000, len(other_set)))
testing_set = torch.utils.data.Subset(other_set, torch.arange(1000))

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True
)
validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=BATCH_SIZE, shuffle=False
)
testing_loader = torch.utils.data.DataLoader(
    testing_set, batch_size=BATCH_SIZE, shuffle=False
)

## Custom CNN

In [ ]:
encoder_decoder_model = ednet4()

In [ ]:
# Defining the loss function (multiclass classification)
loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(encoder_decoder_model.parameters(), lr=0.25, momentum=0.75)

In [ ]:
trainer_ed = Trainer(
    model=encoder_decoder_model,
    criterion=loss_func,
    optimizer=optimizer,
    train_loader=training_loader,
    valid_loader=validation_loader,
)

trainer_ed.set_scheduler(
    scheduler_class=torch.optim.lr_scheduler.StepLR,
    scheduler_params={
        "step_size": 3,
        "gamma": 0.1,
    },
    level="epoch")

In [ ]:
trainer_ed.run(
    epochs=1,
    seed=42,
    enable_tqdm=True,
)

Epoch 1/1 [Validation]: 100%|██████████| 71/71 [00:02<00:00, 30.75batches/s, loss=(1.7665, 1.3197), acc=(0.3433, 0.5284)]


## ResNet - 20 layers

In [ ]:
resnet_20 = resnet20()

In [ ]:
# Defining the loss function (multiclass classification)
loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(resnet_20.parameters(), lr=0.25, momentum=0.75)

In [ ]:
trainer_resnet = Trainer(
    model=resnet_20,
    criterion=loss_func,
    optimizer=optimizer,
    train_loader=training_loader,
    valid_loader=validation_loader,
)

trainer_resnet.set_scheduler(
    scheduler_class=torch.optim.lr_scheduler.StepLR,
    scheduler_params={
        "step_size": 3,
        "gamma": 0.1,
    },
    level="epoch")

In [ ]:
trainer_resnet.run(
    epochs=10,
    seed=42,
    enable_tqdm=True,
)

Epoch 10/10 [Validation]: 100%|██████████| 71/71 [00:02<00:00, 25.37batches/s, loss=(0.5806, 0.6088), acc=(0.7969, 0.7942)]
